### Example program

- From *Kernel-based Detection of Coincidentally Correct Test Cases to Improve Fault Localization Effectiveness*

```c
 e1: int Sample(int a, int b, int c) {
 e2:    int rsum, rdiv, result, rlog=0;
 e3:    result = 0;
 e4:    rdiv = 1;
 e5:    rsum = a + b;
 e6:    if ((a > 0) && (b > 0))
 e7:        rdiv = a / b;
 e8:    rmax = b;
 e9:    if (a > b)
e10:        rmax = b; // Correct: rmax = a;
e11:    if (c == 1)
e12:        result = rsum;
e13:    if (c == 2)
e14:        result = rdiv;
e15:    if (c == 3)
e16:        result = rmax;
e17:    return result;
     }
```

In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
from factorgraph import FactorNode, VariableNode, FactorGraph


# Add example bug execution data

In [3]:
idxs = [
    "e6",
    "e7",
    "e8",
    "e9",
    "e10",
    "e11",
    "e12",
    "e13",
    "e14",
    "e15",
    "e16",
    "e17",
]

exec1_et = [0, 1, 2, 3, 4, 5, 7, 9, 10, 11]
exec2_et = [0, 2, 3, 4, 5, 7, 8, 9, 11]
exec2_et_2 = [0, 2, 3, 4, 5, 7, 8, 9, 11]
exec3_et = [0, 2, 3, 5, 7, 9, 10, 11]
exec4_et = [0, 1, 2, 3, 4, 5, 6, 7, 9, 11]
exec5_et = [0, 1, 2, 3, 5, 7, 9, 10, 11]

execs = [exec1_et, exec2_et, exec2_et_2, exec3_et, exec4_et, exec5_et]

results = [1, 0, 0, 0, 0, 0]

exec_names = [    
    "exec1_et",
    "exec2_et",
    "exec2_et_2",
    "exec3_et",
    "exec4_et",
    "exec5_et",
]

# Create factor graph

## Original variable nodes

In [7]:
variable_list = [VariableNode(idx) for idx in idxs]

for exec_name, result in zip(exec_names, results):
    trnode = VariableNode(f"TR_{exec_name}")
    trnode.val = result
    trnode.fix()
    variable_list.append(trnode)


## Add variable nodes representing whether the faulty element is covered by the execution

In [8]:
for exec_name in zip(exec_names, results):
    covnode = VariableNode(f"Cov_{exec_name}")
    variable_list.append(covnode)

In [9]:
def rel_tr_cov(tr, cov):
    if tr and cov:
        return 1
    elif tr:
        return 0
    elif cov:
        return 0.05
    else:
        return 0.95


scope = {"rel_tr_cov": rel_tr_cov}
factor_list = []
for exec_name in exec_names:
    factor_node = FactorNode(
        f"R_{exec_name}", f"rel_tr_cov(V[TR_{exec_name}], V[Cov_{exec_name}])"
    )
    factor_list.append(factor_node)


## Update factor nodes and generate factor graph

In [ ]:
def prob_succ(elem_faulty):
    # This one is ad-hoc one.
    num_f = sum(elem_faulty)
    num_c = len(elem_faulty) - num_f
    return 0.01 ** (num_f / 1.1 ** num_c) 

def prob_match(result, elem_faulty):
    return 1 - prob_succ(elem_faulty) if result else prob_succ(elem_faulty)

scope["prob_succ"] = prob_succ
scope["prob_match"] = prob_match


for exec_name, et in zip(exec_names, execs):
    covered_elems = [idxs[i] for i in et]
    param_str = ", ".join([f"V[{elem}]" for elem in covered_elems])
    factor_node = FactorNode(
        exec_name, f"prob_match(V[{exec_name}], [{param_str}])")
    factor_list.append(factor_node)

factor_graph = FactorGraph(variable_list, factor_list)
print(factor_graph)